In [37]:
from asammdf import MDF, Signal
from asammdf.blocks.types import DbcFileType, BusType, StrPath
from pathlib import Path
import requests
from datetime import datetime, timedelta
from time import sleep
import json
from collections.abc import Iterable
from collections import defaultdict
import cantools
import numpy as np
import pandas as pd
from sending import *
import pickle

d65_onedrive_files = Path(
    r"/mnt/c/Users/CARAL/Epiroc/Rig Crew - Private - General/5. Testing/CANEdge"
)
lower_files = list((d65_onedrive_files / "Lower").rglob('*.[mM][fF]4'))
upper_files = list((d65_onedrive_files / "Upper").rglob('*.[mM][fF]4'))
print(f'Found {len(lower_files)} MF4 files in lower.')
print(f'Found {len(upper_files)} MF4 files in upper.')

Found 379 MF4 files in lower.
Found 360 MF4 files in upper.


In [39]:
with open('d65_canedge_file_data.pkl', 'rb') as f:
    df = pickle.load(f)

def parse_file(group: str, files: list[Path]):
    print(f"{group} files...")
    _count = len(files)
    print(f" Processing {0:4d}/{_count:4d} [{0:3d}%] ...", end='\r', flush=True)
    for i, file in enumerate(files):
        if not file.exists():
            print(f"File {file} does not exist.")
            df.drop(df[df['File'] == str(file)].index, inplace=True)
            continue
    
        if str(file) in df['File'].values:
            continue
        try:
            with MDF(file, process_bus_logging=False) as mdf:
                _disp = "/".join(str(file).split('/')[str(file).split('/').index(group):])
                print(f" Processing {i:4d}/{_count:4d} [{int(round(100*i/_count)):3d}%] .../{_disp}", end='\r', flush=True)
                start_date = mdf.start_time
                end_date = datetime.fromtimestamp(0, tz=start_date.tzinfo)  # Initialize to epoch in the same timezone
                for ch in mdf.iter_channels():
                    if len(ch.timestamps):
                        if (end_date + timedelta(seconds=ch.timestamps[-1])) > end_date:
                            end_date = mdf.start_time + timedelta(seconds=ch.timestamps[-1])
                
                df.loc[len(df)] = [group, str(file), start_date.isoformat(), end_date.isoformat()]
            
        except Exception as e:
            print(f"Error processing {file}: {e}")
    
    print (f"{group} files ... Complete!", flush=True)

parse_file('Lower', lower_files)
parse_file('Upper', upper_files)

Lower files...
Lower files ... Complete!0%] ...
Upper files...
Lower files ... Complete!0%] ...
Upper files...
Upper files ... Complete!0%] ...
Upper files ... Complete!


In [35]:
df.head()
df.tail()

,Group,File,Start Time,End Time
734,Upper,/mnt/c/Users/CARAL/Epiroc/Rig Crew - Private -...,2025-07-16T17:53:21+00:00,2025-07-16T17:57:38.647150+00:00
735,Upper,/mnt/c/Users/CARAL/Epiroc/Rig Crew - Private -...,2025-07-16T17:57:45+00:00,2025-07-16T17:59:59.372650+00:00
736,Upper,/mnt/c/Users/CARAL/Epiroc/Rig Crew - Private -...,2025-07-16T17:57:45+00:00,2025-07-16T18:03:48.200950+00:00
737,Upper,/mnt/c/Users/CARAL/Epiroc/Rig Crew - Private -...,2025-07-16T18:04:12+00:00,2025-07-16T18:10:21.469750+00:00
738,Upper,/mnt/c/Users/CARAL/Epiroc/Rig Crew - Private -...,2025-07-16T18:21:34+00:00,2025-07-16T18:28:09.375100+00:00


In [36]:
import pickle

# Pickle the dataframe to a file
with open('d65_canedge_file_data.pkl', 'wb') as f:
    pickle.dump(df, f)

print('DataFrame pickled to mf4_times_df.pkl')

DataFrame pickled to mf4_times_df.pkl


In [ ]:
datetime.now().isoformat()

'Lower/5A72CE4C/00001159/00000017.MF4'

In [43]:
datetime.fromisoformat(df.loc[df['File'] == str(lower_files[10])]['End Time'].values[0])

datetime.datetime(2025, 6, 21, 13, 29, 59, 984500, tzinfo=datetime.timezone.utc)